In [29]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import hddm
import os
from datetime import datetime
from timeit import default_timer as timer
from datetime import timedelta
from scipy.stats import spearmanr
import statistics
from zepid.graphics import EffectMeasurePlot
import pingouin as pg
from pymer4.models import Lmer

plt.close('all')
beh_path = ('/Users/nagrodzkij/data/angry/')
input_path = ('/Users/nagrodzkij/data/angry/input/')
output_path = ('/Users/nagrodzkij/data/angry/output/')

FaceScores = pd.read_csv(input_path+'/demog/FaceScores.csv',index_col=[0])

prev_depressed = pd.read_csv(input_path+'/demog/previously_depressed.csv', names=['0'])
list_depressed = prev_depressed['0'].to_list()

demog = pd.read_csv(output_path+'table_demog_byccid.csv')

################################

os.chdir(output_path+'hddm/')
hddm_path=(output_path+'hddm/')
accuracy_coding_path = hddm_path+'accuracy_coding/'
accuracy_coding_models_path=accuracy_coding_path+'models/'

data = hddm.load_csv(output_path+'data_emoFace_excl.csv')
demog = pd.read_csv(output_path+'table_demog_withedu_byccid.csv',index_col=[0])
age = demog.age
FaceScores = pd.read_csv(input_path+'/demog/FaceScores.csv',index_col=[0])

# Functions for correlations

def mode(arr):
    if len(arr) == 0:
        return []

    frequencies = {}

    for num in arr:
        frequencies[num] = frequencies.get(num,0) + 1

    mode = max([value for value in frequencies.values()])

    modes = []

    for key in frequencies.keys():
        if frequencies[key] == mode:
            modes.append(key)

    return modes

def bayesian_probability(posterior_distribution):
    ''' Calculates probability that a posterior distribution is different from 0
    Parameters
    --------------------
    posterior_distribution : iterable object such as a list or np.array

    Returns
    --------------------
    perc : a float (in percent), percentage probability that the posterior distribution is different from 0
    '''
    j = posterior_distribution
    if statistics.mean(j) > 0:
        corr_values = [i for i in j if i > 0]
        perc = len(corr_values) / len(j) * 100
    else:
        corr_values = [i for i in j if i < 0]
        perc = len(corr_values) / len(j) * 100

    return perc

def pearsonr_ci(x,y,alpha=0.05):
    ''' calculate Pearson correlation along with the confidence interval using scipy and numpy
    Parameters
    ----------
    x, y : iterable object such as a list or np.array
      Input for correlation calculation
    alpha : float
      Significance level. 0.05 by default
    Returns
    -------
    r : float
      Pearson's correlation coefficient
    pval : float
      The corresponding p value
    lo, hi : float
      The lower and upper bound of confidence intervals
    '''

    r, p = statistics.pearsonr(x,y)
    r_z = np.arctanh(r)
    se = 1/np.sqrt(x.size-3)
    z = statistics.norm.ppf(1-alpha/2)
    lo_z, hi_z = r_z-z*se, r_z+z*se
    lo, hi = np.tanh((lo_z, hi_z))
    return r, p, lo, hi

def ddm_param_corr(ddm_param,covariate,ddm,cov,save):
    ''' calculate Spearmann's rank correlation coefficient between DDM parameter and any covariate (must be 1 value per participant)
    Parameters
    ----------
    ddm_param : dataframe with any number of values of DDM parameters per participant
    covariate : iterable object such as a list or np.array
      Input for correlation calculation
    ddm : name of ddm_param being correlated
    cov : name of covariate as will be visible in the saved file (e.g. when name = 'age', saved file is rho_vsens_age.csv)
    save : whether you want to save the output; =1 will save the output in a csv file

    Returns
    -------

    dt : dictionary of values of rho, p-values, and significance levels
    df : dataframe of values of rho, p-vales, and significance levels
    '''

    dt = {'rho': [], 'p-value': [], 'sig': []}

    if len(ddm_param.columns[0][:6]) > 1:
        ddm_param_ind = ddm_param.transpose()
        check = 1
    elif len(ddm_param.index[0][:6]) > 1:
        ddm_param_ind = ddm_param
        check = 1
    else:
        check = 0

    assert check == 1, 'Incorrect input type' # Checks if v_sens is of the correct input type

    for i in range(0, len(ddm_param_ind.columns)):
        ddm_param_run = ddm_param_ind.iloc[:, [i]]
        data1 = ddm_param_run
        data2 = covariate

        coef, p = spearmanr(data1, data2)
        dt['rho'].append(coef)
        dt['p-value'].append(p)

        if p > 0.05:
            dt['sig'].append('ns')
        elif 0.01 < p <= 0.05:
            dt['sig'].append('*')
        elif 0.001 < p <= 0.01:
            dt['sig'].append('**')
        elif p <= 0.001:
            dt['sig'].append('***')

    df = pd.DataFrame.from_dict(dt)
    if save == 1:
        df.to_csv('/Users/nagrodzkij/data/driftmodel/emo_recog_hddm/rho_%s_%s.csv' %(ddm,cov))

    return dt, df

def trim_to_hdi(df,perc):
    ''' Trim a dataframe by a specified amount on each side to obtain a Bayesian high density interval from rho values
    Parameters
    ----------
    df : dataframe containing column 'rho' with values of rho
    perc : percentage specifying the extent of the high-density interval, e.g. for a 95% HDI specify 95

    Returns
    -------
    df_trimmed : dataframe containing trimmed values
    '''

    assert perc <=100, 'Specify extent of HDI as a percentage e.g. 95 for a 95% HDI'
    assert perc > 70, 'Specify extent of HDI as a percentage e.g. 95 for a 95% HDI'
    each_side = (100 - perc)/2
    df_sorted = np.array(sorted(df['rho']))
    trim = int(each_side*df_sorted.size/100)
    df_trimmed = df_sorted[trim:-trim]

    return df_trimmed

def add_values_in_dict(dict, key, list_of_values):
    ''' Append multiple values to a key in
        the given dictionary '''
    if key not in dict:
        dict[key] = list_of_values
    else:
        dict[key].extend(list_of_values)
    return dict

# Functions for fitting DDM

def get_combined_traces(models):
    traces = []
    for i in range(len(models)):
        m = models[i]
        trace = m.get_traces()
        trace['chain'] = i + 1
        traces.append(trace)

    return traces

def run_hddm_stimcoding(id, data2fit, dependency, folder, name, incl, split_by,group_nodes):
    print()
    print('running model %i' % id)

    if split_by == ():
        dbname = '%s/models/%s_%i.db' % (folder, name, id)
        mname = '%s/models/%s_%i' % (folder, name, id)
        m = hddm.HDDMStimCoding(data2fit,
                                p_outlier=0.05,
                                include=incl,
                                stim_col='stim',
                                depends_on=dependency,
                                group_only_nodes='%s' % group_nodes
                                )

        m.find_starting_values()
        m.sample(10000, burn=3000, thin=5, dbname=dbname, db='pickle')

        m.save(mname)

        return m

    elif split_by == 'v':
        dbname = '%s/models/%s_%i.db' % (folder, name, id)
        mname = '%s/models/%s_%i' % (folder, name, id)
        m = hddm.HDDMStimCoding(data2fit,
                                p_outlier=0.05,
                                include=incl,
                                split_param='v',
                                stim_col='stim',
                                depends_on=dependency,
                                group_only_nodes='%s' % group_nodes
                                )

        m.find_starting_values()
        m.sample(10000, burn=3000, thin=5, dbname=dbname, db='pickle')

        m.save(mname)

        return m

    elif split_by == 'z':
        dbname = '%s/models/%s_%i.db' % (folder, name, id)
        mname = '%s/models/%s_%i' % (folder, name, id)
        m = hddm.HDDMStimCoding(data2fit,
                                p_outlier=0.05,
                                include=incl,
                                split_param='z',
                                stim_col='stim',
                                depends_on=dependency,
                                )

        m.find_starting_values()
        m.sample(10000, burn=3000, thin=5, dbname=dbname, db='pickle')
        m.save(mname)

        return m

def run_hddm_stimcoding_multiple(data2fit, dependency, folder, name, incl, split_by, num_chains,group_nodes):
    for i in np.arange(3, num_chains+1):
        id = i
        run_hddm_stimcoding(id, data2fit, dependency, folder, name, incl, split_by,group_nodes)

    return

def hddm_save_trace(name, save=1):
    models = []
    for ident in np.arange(1, 5):
        mname = accuracy_coding_path+'models/'+name+'_'+str(ident)
        m_stim = hddm.load(mname)
        models.append(m_stim)

        params = ('a', 't', 'v')
        for param in params:
            plot = m_stim.plot_posteriors(param)
            plot = plt.gcf()
            plot.savefig(accuracy_coding_models_path+name+'_'+str(ident)+param+'.jpg')
        plt.close('all')

    # rhat = hddm.analyze.gelman_rubin(models)  # get the R hat information

    if save:
        traces = get_combined_traces(models)
        traces = pd.concat(traces)
        traces.to_csv(accuracy_coding_models_path+name+'_trace.csv')

    # comb = kabuki.utils.concat_models(models)
    return models

def plot_dep_vcond(name):
    m = hddm.load(name)
    v_AngryMale, v_AngryFemale, v_NeutralMale, v_NeutralFemale = \
        m.nodes_db.node[['v(Angry.0)', 'v(Angry.1)', 'v(Neutral.0)', 'v(Neutral.1)']]
    hddm.analyze.plot_posterior_nodes([v_AngryMale, v_AngryFemale, v_NeutralMale, v_NeutralFemale])
    plt.xlabel('Drift rate')
    plt.ylabel('Posterior probability')
    plt.title(f'Posterior of drift-rate group means. Model ' + name)
    plot = plt.gcf()
    plot.savefig(name + '_plot.jpg')
    plt.show()

    return plot, m

def plot_dics(name):
    dics = hddm.load_csv('%s_dics.csv' % (name))

    # This ensures the figure size matches the number of models being plotted
    n = len(dics) + 1
    pix = 0.3 * n

    dics.plot(x='id', y='dic', marker='o', legend=False)
    fig = plt.gcf()
    fig.set_size_inches(pix, 5)

    # Setting axes and title
    plt.xlim([0, n])
    plt.xticks(np.arange(0, n, 1.0))
    plt.title(
        'Graph of DIC by model number\n'
        # '(best model is marked in red, the second and third best models are marked in black)'
        )
    plt.xlabel('Model ID')
    plt.ylabel('DIC')

    # Make a grid
    minor_ticks = np.arange(1.5,29,1)
    major_ticks = np.arange(0,31,1)

    ax = plt.gca()
    ax.set_xticks(minor_ticks, minor=True)
    ax.set_xticks(major_ticks)

    ax.grid(which='minor',alpha=0.6)

    # Marking the top 3 models
    dics_sorted = dics.sort_values('dic', ascending=True)
    plt.ylim([int(dics_sorted.iloc[0]['dic']) * 0.7, int(dics_sorted.iloc[len(dics) - 1]['dic']) * 1.1])

    for iBest in np.arange(0, 3):
        best_iBest = int(dics_sorted.iloc[iBest]['dic'])
        bestmodel_iBest = int(dics_sorted.iloc[iBest]['id'])
        fig = plt.gcf()
        if iBest == 0:
            plt.plot(bestmodel_iBest, best_iBest, 'ro')
            plt.annotate("%i" % (bestmodel_iBest), (bestmodel_iBest, best_iBest * 0.8), ha='center')
        else:
            plt.plot(bestmodel_iBest, best_iBest, 'ko')
            plt.annotate("%i" % (bestmodel_iBest), (bestmodel_iBest, best_iBest * 0.8), ha='center')

    # Save
    fig = plt.gcf()
    fig.savefig('testing_all_dic_plot.jpg')
    plt.show()

    return n, pix

def models_stimcoding_from_matrix(data2fit, filename, folder, name, group_nodes):
    """This function runs multiple models specified in a csv file.

        Arguments
        - data2fit: data frame with data
        - filename: name of csv file with matrix
        - name: the required name of the run of models e.g. 'test' or 'final'
        - folder: path to the directory which has the csv file and a folder models in which the models will be saved

        Requirements
        - csv file needs to be created from MASTER_model_matrix (transposed) or from scratch
        - it needs to have a column with model numbers without a heading followed by columns named:
                            "include = 'z' ", "depends_on = {'v':'cond'}",
                            "depends_on = {'v':'stim'}", "depends_on = {'z':'cond'}",
                            "depends_on = {'z':'stim'}", "depends_on = {'a':'cond'}",
                            "depends_on = {'a':'stim'}", "split_param = 'v'",
                            "split_param = 'z'"
        - to indicate a parameter is used, cell value = 1 and to indicate it isn't used leave blank or = 0

        Saves
        - a txt file named report_name.txt which describes the characteristics of each model and DIC
        - models named name_1, name_2, ...
        - a csv file of all DICs by model ID
        """

    # Read file and rename column names to something that will work with the code
    df = pd.read_csv(filename)
    df.rename(columns={"Unnamed: 0": "num", "include = 'z' ": "z", "depends_on = {'v':'cond'}": "v : cond",
                       "depends_on = {'v':'stim'}": "v : stim", "depends_on = {'z':'cond'}": "z : cond",
                       "depends_on = {'z':'stim'}": "z : stim", "depends_on = {'a':'cond'}": "a : cond",
                       "depends_on = {'a':'stim'}": "a : stim", "split_param = 'v'": "split_param = v",
                       "split_param = 'z'": "split_param = z"}, inplace=True)
    df.replace(np.nan, 0, inplace=True)  # Replace all NaN (empty cells) with zeros

    f = open('%s/models/report_%s_dic.txt' % (folder, name), "w+")
    f.write("Log of drift diffusion model fitting\n")
    start = timer()
    f.write(("Script commenced at " + datetime.now().strftime("%d/%m/%Y %H:%M:%S") + "\n\n\n"))
    f.write(("Let's go!\n\n\n"))
    f.close()

    rows = len(df.index)
    dics = {'id': [], 'dic': []}
    for iMat in np.arange(0, rows):
        err = 0
        id = iMat + 1

        # Check if z is included
        if df.iloc[iMat]['z'] == 1:
            incl = 'z'
        else:
            incl = ()

        # Obtain split_param
        split_by = ()
        check_split = df.iloc[iMat, 8:10] == 1
        k_true = list(check_split.loc[check_split == True].index)
        if len(k_true) > 1:
            err = 1
            continue
        elif len(k_true) == 1:
            split_by = '%s' % (k_true[0][-1])
        elif len(k_true) == 0:
            split_by = ()

        # Obtain dependency
        dependency = {}
        check_dep = df.iloc[iMat, 2:8] == 1
        check_true = check_dep.loc[check_dep == True]
        l_true = list(check_true.index)
        params = list()

        for iL in np.arange(0, len(l_true)):
            param = l_true[iL][0]
            if param in params:
                dependency['%s' % (param)] = ['cond', 'stim']
            else:
                params.append(param)
                dependency['%s' % (param)] = '%s' % (l_true[iL][4:])

        # Run the model
        m = run_hddm_stimcoding(id, data2fit, dependency, folder, name, incl, split_by, group_nodes)

        # os.chdir('%s/models' % (folder))
        # m = hddm.load('%s_%i' % (name, id))

        dic = m.dic
        dics['id'].append('%i' % (id))
        dics['dic'].append('%f' % (dic))

        # Prints characteristics of each model in the log file
        f = open('%s/models/report_%s_dic.txt' % (folder, name), "a")
        f.write((f'Model ' + str(id) + "\n"))
        f.write(('    include ' + str(incl) + "\n"))
        f.write(('    split_by ' + str(split_by) + "\n"))
        f.write(('    dependency ' + str(dependency) + "\n"))
        f.write(('    DIC: ' + str(dic) + "\n"))
        if err == 1:
            f.write('Something went wrong! Splitting by more than 1 parameter' + "\n")
        f.write("\n")
        f.close()

    f = open('%s/models/report_%s_dic.txt' % (folder, name), "a")
    f.write(("DDM fitting complete!\n"))
    min_dic = min(dics, key=lambda x: x[1])[1]
    model_min_dic = min(dics, key=lambda x: x[1])[0]
    f.write(("The best fitting model is model " + str(model_min_dic) + " with a DIC of " + str(min_dic) + "\n\n\n"))

    dics_df = pd.DataFrame(dics)
    dics_df.to_csv('%s_dics.csv' % (name), index=False)

    end = timer()
    f.write(("Script finished at " + datetime.now().strftime("%d/%m/%Y %H:%M:%S") + "\n"))
    f.write(("Total runtime: " + str(timedelta(seconds=end - start))))
    f.close()

    return rows, f, dics

def run_hddm(ident, data2fit, dependency, folder, name, sample_no, burn_no, thin_no, participant_model):
    print()
    print('running model %i' % ident)


    dbname = '%smodels/%s_%i.db' % (folder, name, ident)
    mname = '%smodels/%s_%i' % (folder, name, ident)
    m = hddm.HDDM(data2fit,
                            p_outlier=0.05,
                            depends_on=dependency,
                            is_group_model=participant_model,
                            )

    m.find_starting_values()
    m.sample(sample_no, burn=burn_no, thin=thin_no, dbname=dbname, db='pickle')

    m.save(mname)

    return m

def models_from_matrix(data2fit, filename, folder, name, sample_no, burn_no, thin_no,participant_model):
    """This function runs multiple models specified in a csv file.

        Arguments
        - data2fit: data frame with data
        - filename: name of csv file with matrix
        - name: the required name of the run of models e.g. 'test' or 'final'
        - folder: path to the directory which has the csv file and a folder models in which the models will be saved

        Requirements
        - csv file needs to be created from MASTER_model_matrix (transposed) or from scratch
        - it needs to have a column with model numbers without a heading followed by columns named:
                            "depends_on = {'v':'cond'}",
                            "depends_on = {'z':'cond'}",
                            "depends_on = {'a':'cond'}",
        - to indicate a parameter is used, cell value = 1 and to indicate it isn't used leave blank or = 0

        Saves
        - a txt file named report_name.txt which describes the characteristics of each model and DIC
        - models named name_1, name_2, ...
        - a csv file of all DICs by model ID
        """

    # Read file and rename column names to something that will work with the code
    df = pd.read_csv(filename)
    df.rename(columns={"Unnamed: 0": "num", "depends_on = {'v':'cond'}": "v : cond",
                       "depends_on = {'z':'cond'}": "z : cond",
                       "depends_on = {'a':'cond'}": "a : cond",
                       "depends_on = {'t':'cond'}": "t : cond"
                       },
              inplace=True)
    df.replace(np.nan, 0, inplace=True)  # Replace all NaN (empty cells) with zeros

    f = open('%smodels/report_%s_dic.txt' % (folder, name), "w+")
    f.write("Log of drift diffusion model fitting\n")
    start = timer()
    f.write("Script commenced at " + datetime.now().strftime("%d/%m/%Y %H:%M:%S") + "\n\n\n")
    f.write("Let's go!\n\n\n")
    f.close()

    rows = len(df.index)
    dics = {'id': [], 'dic': []}
    for iMat in np.arange(0, rows):
        models=[]
        err = 0
        ident = iMat + 1

        # Obtain dependency
        dependency = {}
        check_dep = df.iloc[iMat, 1:6] == 1
        check_true = check_dep.loc[check_dep == True]
        l_true = list(check_true.index)
        params = list()

        for iL in np.arange(0, len(l_true)):
            param = l_true[iL][0]
            if param in params:
                dependency['%s' % (param)] = ['cond', 'stim']
            else:
                params.append(param)
                dependency['%s' % (param)] = '%s' % (l_true[iL][4:])

        # Run the model
        m = run_hddm(ident, data2fit, dependency, folder, name, sample_no, burn_no, thin_no,participant_model)

        # os.chdir('%s/models' % (folder))
        # m = hddm.load('%s_%i' % (name, id))

        dic = m.dic
        dics['id'].append('%i' % (ident))
        dics['dic'].append('%f' % (dic))

        # Prints characteristics of each model in the log file
        f = open('%smodels/report_%s_dic.txt' % (folder, name), "a")
        f.write((f'Model ' + str(ident) + "\n"))
        f.write(('    dependency ' + str(dependency) + "\n"))
        f.write(('    DIC: ' + str(dic) + "\n"))
        if err == 1:
            f.write('Something went wrong! Splitting by more than 1 parameter' + "\n")
        f.write("\n")
        f.close()

    f = open('%smodels/report_%s_dic.txt' % (folder, name), "a")
    f.write(("DDM fitting complete!\n"))
    min_dic = min(dics, key=lambda x: x[1])[1]
    model_min_dic = min(dics, key=lambda x: x[1])[0]
    f.write(("The best fitting model is model " + str(model_min_dic) + " with a DIC of " + str(min_dic) + "\n\n\n"))

    dics_df = pd.DataFrame(dics)
    dics_df.to_csv(accuracy_coding_path+'/models/'+name+'_dics.csv', index=False)

    hddm_save_trace(name,save=1)

    end = timer()
    f.write(("Script finished at " + datetime.now().strftime("%d/%m/%Y %H:%M:%S") + "\n"))
    f.write(("Total runtime: " + str(timedelta(seconds=end - start))))
    f.close()

    return rows, f, dics

def run_hddm_multiple(data2fit, dependency, folder, name, num_chains, sample_no, burn_no, thin_no,participant_model):
    for i in np.arange(1, num_chains+1):
        ident = i
        run_hddm(ident, data2fit, dependency, folder, name, sample_no, burn_no, thin_no,participant_model)
    return

In [30]:
data = data[data["rt"] > 0.25]

# Format the data frames correctly for HDDM modelling
data.rename(
    columns={"stim_col": "stim", "condition": "cond"}, inplace=True)

data_hddm = data.drop(columns=['sot','duration','button','trial'])

# Create data frame in which the 'response' column indicates if response is correct or incorrect
response_acc = (data['response'] == data['stim']) + 0
data_acc = pd.DataFrame()
data_acc['rt'] = data['rt']
data_acc['response'] = response_acc
data_acc['subj_idx'] = data['subj_idx']
data_acc['cond'] = data['cond']

data_hddm.to_csv(output_path+'data_emoFace_hddm.csv')
data_acc.to_csv(output_path+'accdata_emoFace_hddm.csv')

In [31]:
# Running group-only accuracy model for model selection

models_from_matrix(data2fit=data_acc,filename=input_path+'/matrices/accuracy_model_matrix1.csv',folder=accuracy_coding_path,name='accuracy_testing_all1',sample_no=5000,burn_no=1000,thin_no=2,participant_model=False)

# Running winning model with participant nodes and multiple chains

run_hddm_multiple(data2fit=data_acc,dependency={'v': 'cond'},folder=accuracy_coding_path,
                  name='accuracy_participant1',num_chains=5,sample_no=10000,burn_no=2000,thin_no=5,
                  participant_model=True)

hddm_save_trace('accuracy_participant1',save=1)




running model 1
-3489.992061015614
-3489.98955060257
 [-----------------100%-----------------] 5000 of 5000 complete in 506.3 sec
running model 2
-7200.526834057502
-7200.515628240032
 [-----------------100%-----------------] 5000 of 5000 complete in 490.6 sec
running model 3
-3468.7378961733643
-3468.7376386515452
 [-----------------100%-----------------] 5000 of 5000 complete in 512.9 sec
running model 4
-3488.2148806672494
-3488.2038800707187
 [-----------------100%-----------------] 5000 of 5000 complete in 558.3 secPlotting a
Plotting t
Plotting v
Plotting a
Plotting t
Plotting v(Angry)
Plotting v(Neutral)
Plotting a(Angry)
Plotting a(Neutral)
Plotting t
Plotting v
Plotting a
Plotting t(Angry)
Plotting t(Neutral)
Plotting v

running model 1


/Users/nagrodzkij/opt/anaconda3/envs/untitled/lib/python3.6/site-packages/scipy/optimize/optimize.py:1985: RuntimeWarning: invalid value encountered in double_scalars
  tmp2 = (x - v) * (fx - fw)


 [-----------------100%-----------------] 10001 of 10000 complete in 6953.1 sec-----------------81%----------        ] 8130 of 10000 complete in 5693.4 sec
running model 2
 [-----------------100%-----------------] 10001 of 10000 complete in 7035.0 sec-----------------77%---------         ] 7751 of 10000 complete in 5440.0 sec
running model 3
 [-----------------100%-----------------] 10001 of 10000 complete in 6727.3 sec
running model 4
 [-----------------100%-----------------] 10001 of 10000 complete in 6681.7 sec
running model 5
 [-----------------100%-----------------] 10001 of 10000 complete in 6668.6 secPlotting a
Plotting t
Plotting v(Angry)
Plotting v(Neutral)
Plotting a
Plotting t
Plotting v(Angry)
Plotting v(Neutral)
Plotting a
Plotting t
Plotting v(Angry)
Plotting v(Neutral)
Plotting a
Plotting t
Plotting v(Angry)
Plotting v(Neutral)


In [35]:
# GENERATING A DATAFRAME WITH MEAN V_ANGRY AND MEAN V_NETURAL PER PARTICIPANT

traces = pd.read_csv(accuracy_coding_models_path+'accuracy_participant1_trace.csv',index_col=0)

ccid = pd.unique(data.subj_idx)

# Extract column names which refer to v_angry from big traces dataframe
col_names_angry = [col for col in traces.columns if "v_subj(Angry)" in col]
col_names_neutral = [col for col in traces.columns if "v_subj(Neutral)" in col]

# Extract the values of v_angry and v_neutral only
params_angry = traces.loc[:,col_names_angry]
params_neutral = traces.loc[:,col_names_neutral]

# Calculate the mean of v_angry and v_neutral for each participant
params_angry_mean = params_angry.mean(axis=0)
params_neutral_mean = params_neutral.mean(axis=0)

# Shorten the names of the row names to include only ccid
indexes = []
for ix in params_angry_mean.index:
    ix = ix[14:]
    indexes.append(ix)
params_angry_mean.index = indexes

indexes = []
for ix in params_neutral_mean.index:
    ix = ix[16:]
    indexes.append(ix)
params_neutral_mean.index = indexes

d = {'v_angry': params_angry_mean, 'v_neutral': params_neutral_mean}

params_v_mean = pd.DataFrame(d)

if '610496' in params_v_mean.index:
    params_v_mean = params_v_mean.drop('610496')

params_v_mean.to_csv(output_path+'params_v_byparticip.csv')